<a href="https://colab.research.google.com/github/sarang096/Genai_Assignment_/blob/main/PES2UG23CS5333_3_Advanced_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 6.4 MB/s eta 0:00:00
Enter your Groq API Key: ··········


In [2]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


In [3]:
# 2. CoT Prompt (Magic Phrase)
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the new tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


In [4]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a Child Psychologist, I would recommend **Solution 2: Make a "Veggie Garden" in the House** as the most sustainable option for encouraging a 5-year-old to eat vegetables. Here's why:

1. **Empowerment**: By allowing your child to participate in the process of planting and maintaining a "veggie garden," you are giving them a sense of ownership and control over the food they eat. This can foster a sense of empowerment and motivation to try new vegetables.
2. **Education**: This approach introduces your child to the concept of where food comes from and the process of growing their own food. This can lead to a greater appreciation and understanding of the value of vegetables.
3. **Personal connection**: As the vegetables grow, your child will have a personal connection to the food they're consuming, which can make them more invested in eating it.
4. **Long-term impact**: This approach can have a long-term impact on your child's eating habits, as the

In [6]:
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
"Echoes of Eternity" is a mind-bending, heart-pounding thriller that combines the scientific marvels of time travel with the passionate stakes of romance and the terror of a supernatural force. When brilliant physicist Emma Taylor discovers a way to communicate with her past self through a mysterious device, she inadvertently unleashes a vengeful entity known only as "The Shadow" - a malevolent presence that feeds on human emotions and will stop at nothing to claim her soul. As Emma navigates her complex web of past relationships, she must also confront the dark entity that's growing stronger with each passing day, threatening to destroy not just her love life, but the very fabric of time itself. With the help of her current love interest, a charming and resourceful historian named Max, Emma must outwit The Shadow, repair the damage to the timeline, and confront the ultimate question: can she change her past without sacrificing the love she's foun